# DRAFT - SONATA Spatial Tool Demonstration Notebook

<img src="img/logo-SONATA-jpeg.jpg" width="300">

_Frederik Priem (frederik.priem@vito.be)_

_version: xxxx/xx/xx_

## Content
- [About the SONATA spatial tool](#about-the-sonata-spatial-tool)
- [About the SONATA project](#about-the-sonata-project)
- [Goal of the demonstration](#goal-of-the-demonstration)
- [Scope of the demonstration](#scope-of-the-demonstration)
- [Using the SONATA spatial tool](#using-the-sonata-spatial-tool)
    - [Defining the optimization problem](#defining-the-optimization-problem)
        - [Spatial layers](#spatial-layers)
        - [The model space](#the-model-space)
        - [Static/file-based layers](#staticfilebased-layers)
        - [Dynamic/function-based layers](#dynamicfunctionbased-layers)
        - [Objective criteria](#objective-criteria)
        - [Decision criteria](#decision-criteria)
        - [Demand](#demand)
        - [Algorithm parameters](#algorithm-parameters)
    - [Example use case](#example-use-case)
    - [Initiating the optimizer instance](#initiating-the-optimizer-instance)
    - [Running the optimization process](#running-the-optimization-process)
    - [Performing analysis on/with the calibrated optimizer](#performing-analysis-onwith-the-calibrated-optimizer)
        - [Learning curve](#learning-curve)
        - [Best per-criterion/weighted solution](#best-percriterionweighted-solution)
        - [Pareto frequency map](#pareto-frequency-map)
        
## About the SONATA spatial tool
The __SONATA spatial tool__ is a (not yet published) Python library for __solving single- or multi-objective spatial optimization problems__ that can be represented in a __raster GIS environment__. The spatial tool draws on AI-powered [_Metaheuristic Optimization_](https://en.wikipedia.org/wiki/Metaheuristic), using an [_Evolutionary Algorithm_](https://en.wikipedia.org/wiki/Evolutionary_algorithm), and more precisely [_Differential Evolution_](https://en.wikipedia.org/wiki/Differential_evolution), to help explore complex search spaces efficiently and generate (approximated) optimal solutions. If the considered optimization problem is multi-objective, i.e., having more than 1 objective criterion, then the spatial tool will not yield a single optimal solution but a collection of non-dominated solutions forming a [Pareto front](https://en.wikipedia.org/wiki/Pareto_front).

Note that the development of the spatial tool is still in an __experimental stage__ and is not yet deemed suitable for operational use. The code examples shown in this notebook may not be representative for how the spatial tool will work once published.

## About the SONATA project
The spatial tool is currently being developed in the frame of the HEU [__SONATA__](https://sonata-nbs.com/) project, to help find optimal locations for __Nature-based Solutions__, like semi-natural grasslands and flower strips, with the goal to __maximize useful ecosystem services__ in the [Vojvodina region](https://en.wikipedia.org/wiki/Vojvodina) in Serbia. The ecosystem services of interest include _crop yield, pollinator diversity and carbon sequestration_. However, the conceptual and code framework on which the spatial tool is based is __flexible__ and essentially __domain-independent__. So the spatial tool can easily be applied for other types of optimization problems as well.

## Goal of the demonstration
The first goal of this demonstration is to __illustrate__ the workings of the spatial tool to an audience having experience in (spatial) optimization or related domains, or that stands to benefit from its use to support their activities. Secondly, we would also very much like to receive your __feedback__ on the spatial tool, which can help us guide further development and improve the quality of the tool. See please do _share your experiences and thoughts_ on this tool with the member(s) of the development team performing the demonstration. You can also reach us by e-mail ([frederik.priem@vito.be](frederik.priem@vito.be)).

## Scope of the demonstration
Over the next parts of this notebook, we will cover the __basic use of the spatial tool__. We will firstly clarify what are the __key inputs__ of the spatial tool. This will allow you to define your spatial optimization problem and initiate an _Optimizer instance_ containing the core functionality of the spatial tool. Then, we will go through a __prepared example use case__, for which we have included several raster layers covering a small area in Vojvodina. This will allow you to experience a _complete yet simplified optimization workflow using the spatial tool_. If you want to, you'll also be able to tinker with the given optimization problem definition and/or the spatial tool parameters.

## Using the SONATA spatial tool
We can consider __4 steps__ in the use of the spatial tool, that must be covered in this order:

1. Defining the optimization problem
2. Initiating the optimizer instance
3. Running the optimization process
4. Performing analysis on/with the calibrated optimizer

Over the following sections of this notebook, we will cover each of these steps with an explanation and a code example.

### Defining the optimization problem
The first thing to do when using the __SONATA spatial tool__ is _defining the optimization problem_ you want to tackle. Practically this means you'll have to specify _in code_ the key components of the spatial optimization problem:

1. Spatial layers
2. Objective criteria
3. Decision criteria
4. Demand
5. [Optional] Algorithm parameters
    
Defining a real spatial optimization problem will typically entail quite a few layers, objective/decision criteria (with or without spatial constraints) and custom functions. Generally, this will be the part in which users invest most coding effort. The __spatial tool__ does provide some support functions to ease optimization problem definition. The support functions are especially helpful to define custom layer/criterion functions using fewer lines of code.

To circumvent having to discuss a full optimization problem definition in this demonstration, we will instead _illustrate each of the 5 spatial tool components_ in the above list with a small example. Afterwards, we will continue the demonstration with a _pre-coded example_ of an optimization problem definition, that is included in the scripts called [problem.py](problem.py) and [functions.py](functions.py).

#### Spatial layers
There are 3 types of spatial layers that can be used for the problem definition:

1. The model space
2. Static layers
3. Dynamic/function-based layers

All layers used in an optimization problem definition must be specified using the _Layer_ class, and these Layer objects must be collected and passed to the an _Optimizer_ instance, which will be discussed further on, as _a list object_. Note that during optimization, the layer values will be loaded/updated in the order in which they are specified in this list.

#### The model space
Exactly 1 __model space__ layer must always be included in the collection of layers of an optimization problem. The model space is essentially the _base layer_ describing the geographic space in which optimization will be performed.

Each cell in the model space has a _cover type_, reflecting a certain _land use_ or _land cover_ class. A __label map__ must be provided alongside the model space, to specify both the numerical labels (used in the model space layer array) and alphanumerical labels for each cover type, and to indicate which cover types can occur in the model space. The initial state of the model space must be loaded from a specified _source file_, and the original values will change in the course of optimization.

The model space also serves as a _spatial reference_, because the other layers of the problem definition must have the _same dimensions as the model space layer_. Internally, the spatial tool will perform _dimension checks_ during optimizer instance creation. If a layer doesn't have the same dimensions as the model space, an error will be raised.


In [ ]:
from src.spatial_tool import Layer

layers = []

lmap = {
    'background': 0,
    'pasture': 1,
    'semi-natural grassland': 2,
    'forest': 3,
    'wetland/surface water': 4,
    'cropland': 5
    }

model_space = Layer(
        layer_id='model space',
        source='data/model_space.tif',
        model_space=True,
        label_map=lmap
)

layers.append(model_space)


#### Static/file-based layers
As the name implies, __static or file-based layers__ are layers whose values remain _constant_ throughout the optimization process. These values only have to be loaded once from a specified source file.

By default, layer values are __cached__ into the layer object. This avoids having to repeatedly reload all values, that don't change anyhow, during optimization, which would result in unnecessarly longer runtimes. However, users can turn off caching layer values if they want to. Note that the original values of the earlier discussed model space are also cached by default, to enable quickly resetting the model space.

To save disk space, layers containing float numbers are often converted to integer format, by multiplying the original values with a __scaling factor__ to preserve a certain number of decimals. If the optional _'scale'_ argument is specified, the values from the source file will be divided by the corresponding scale factor when loaded, to obtain the correct data format.


In [ ]:
soc = Layer(
    layer_id='soil organic carbon',
    source='data/soil_organic_carbon.tif',
    scale=10.,  # stored as int, correct format is float with 1 decimal (% of top soil layer)
    cache=True  # default value
)

layers.append(soc)


#### Dynamic/function-based layers
__Dynamic__ or __function-based layers__ are layers that are defined as a function of other layers included in the problem definition. Contrary to the earlier discussed static layers, the values of dynamic layers will change during the course of optimization and will have to be updated regularly.

Dynamic layers are often derived (in part) from the current state of the model space. Dynamic layers can be used to reflect the fact that changing the cover type of a cell will generally impact the probability of subsequent changes in the neigborhood of this cell (i.e., [spatial dependence](https://en.wikipedia.org/wiki/Spatial_analysis)).

To specify a dynamic layer, pass a __LayerFunction__ instance to the 'source' argument of _Layer_. In turn, to create a LayerFunction instance you must specify:

1. a _list of input layer IDs_, referring to _layers defined priorly in the list containing all layers_, whose layer arrays will be collected in a list that will be passed as the first positional argument to the function;
2. a callable _function object_ to be evaluated (either custom or a pre-defined support function) and whose first positional argument must be a list of layer arrays;
3. optionally, a dictionary containing additional _keyword arguments_ with corresponding values to be passed to the function.

For the sake of brevity we will skip the topic of defining _custom layer functions_ (see [functions.py](functions.py) for some examples if you are interested). In the code block below, we instead show two relatively easy examples.

The _first example_ shows the creation of a dynamic layer containing a semi-natural grassland mask (1 if the cell is semi-natural grassland, and 0 otherwise), using the pre-defined support funtion _indicate_ with the _predicate_ set to 'eq' (equals). This mask is derived from the current state of the model space.

The _second example_ shows a dynamic layer defined as the squared transform of the previously defined layer _soil organic carbon_. Here we use the pre-defined support function _power_ with the keyword argument _exponent_ set to '2' to obtain the desired layer function.


In [ ]:
from src.spatial_tool import LayerFunction, indicate, power

layfun_ind_sng = LayerFunction(
    input_layer_ids=['model space'],
    function=indicate,
    kwargs={
        'target': 'semi-natural grassland',
        'predicate': 'eq'
    }
)
ind_sng = Layer(
    layer_id='semi-natural grassland mask',
    source=layfun_ind_sng
)

layers.append(ind_sng)

layfun_soc_sq = LayerFunction(
    input_layer_ids=['soil organic carbon'],
    function=power,
    kwargs={'exponent': 2}
)
soc_sq = Layer(
    layer_id='soil organic carbon squared',
    source=layfun_soc_sq
)

layers.append(layer)


#### Objective criteria
The __objective criteria__ are the measures used to _evaluate the aptness of solutions_ generated by the spatial tool. Without these criteria it would be impossible for the optimizer to learn and produce increasingly better solutions. 

Recall that you can define _several objective criteria_, in which case the optimization problem becomes _multi-objective_ and spatial tool will generate collections of _non-dominated solutions_ instead of single optimal solution. Note that in the frame of SONATA, we want to _maximize 3 key ecosystem services_: 1. crop yield, 2. pollinator diversity and 3. carbon sequestration. In the full optimization problem definition, discussed further on, we will define a separate objective criterion for each of these ecosystem services.

The __objective function__ underlying each objective criterion should ideally be defined using some _expert knowledge_ or _domain-specific insights_ on the optimization problem at hand. The objective functions should also draw on various layers, static and dynamic, defined in the problem definition, to ensure that the optimizer can properly _interact with and learn from changes made in the modelled environment_. The higher the quality of the objective criteria are, the more likely that the spatial tool will generate meaningful results.

Before you can initiate an _ObjectiveCriterion_ instance, you must first create an _ObjectiveFunction_ instance. Similar to the _LayerFunction_ instance, to create an objective function you need a list of input layer IDs, a function and optionally a dictionary of keyword arguments. In the code example below, we use a pre-defined function called 'agricultural_yield', imported from the [functions.py](functions.py) script, and we pass a rather long list of input layers which haven't all been discussed. Consult the [problem.py](problem.py) script for more info.

We then initiate the objective criterion, passing the criterion label 'agricultural yield' and the earlier defined objective function as inputs, as shown in the __code example__ below. Here we also indicate that this particular criterion is to be _maximized_.


In [ ]:
from src.spatial_tool import ObjectiveFunction, ObjectiveCriterion
from functions import agricultural_yield

objective_criteria = []

objfun_ay = ObjectiveFunction(
    function=agricultural_yield,
    input_layer_ids=[
        'pasture',
        'cropland',
        'relative frequency semi-natural grassland',
        'relative frequency wetland/surface water',
        'relative frequency flower strip',
        'sand fraction',
        'soil organic carbon',
        'slope',
        'road access'
    ]
)
objcrit_ay = ObjectiveCriterion(
    criterion_id='agricultural yield',
    objective_function=objfun_ay,
    maximize=True
)

objective_criteria.append(objcrit_ay)


#### Decision criteria
The __decision criteria__ are essentially layers containing _decision values_ that allow the spatial tool to _generate solutions_.

Whereas an objective criterion yields a single number, speaking to the overal quality of an (adjusted) model space, a decision criterion will tell you something about the __potential to optimize the objective criteria__ if a cover type in demand would allocated to an _individual cell_. Hence, we must define a _separate decision criterion per cover type in demand_ (demand will be covered in the next section) and evaluate decision criteria in each cell.

Just like with objective criteria, there is also an __decision function__ underlying each decision criterion. You are quite free to specify the decision function as deemed fit, and it is again advised to employ other layers specified in the optimization problem, particularly those derived from the model space, as input for these functions. The only conditions that must be respected is that the yielded decision values are structured in an array, having the same dimensions as the _model space_, and that the values in the array are _non-negative_, i.e., falling in the _(0, +inf( range_. If a _decision value = 0_ in a certain cell, it means that the corresponding cover type _cannot be allocated_ there.

An important difference compared to objective functions is that decision functions are __partially specified__. This means that each decision function must contain one or more __unspecified coefficients__. Note that the <span style="color:red;">__values of these coefficients are actually what will be optimized__</span> numerically by the algorithm underlying the spatial tool. If the decision functions are defined correctly, this should allow the spatial tool generate increasingly better solutions.

Generally, a decision function could take on the following suggested form:
 
<div align="left" style="font-size: 130%;">
$U_d = \sum_{i=1}^{n} c_{d,i} W_i + c_{d,bias}$
<br>
<br>
$V_{d} = \frac{Z_d}{1 + e^{-U_d}}$
</div>
<br>
    
Here, $V_d$ denotes the decision values for decision criterion $d$, and $c$ the unspecified coefficients. There is one coefficient for each used (static or dynamic) input spatial layer $W$, of which there are $n$, and an additional coefficient for the bias. The logistic transformation of $U$ ensures that the yielded decision values fall in the (0, 1) value range.

> #### Spatial constraints
The factor $Z_d$ in the above equation represents a key component of spatial optimization that hasn't been discussed yet, namely the __spatial constraints__. Spatial constraints are limiting conditions reflecting a certain geographic logic or requirement. In the spatial tool, spatial constraints are enforced via the decision functions and they apply to the cover type associated with the decision criterion $d$. If, for example, you want to exclude certain cells from being converted to the considered cover type, then it would suffice to include into $Z_d$ a mask that renders all decision values in those cells 0. If $m$ separate constraints are defined for a decision criterion $d$, then the global constraints for this criterion are typically obtained by taking the product of all individual constraints:
> <div align="left" style="font-size: 130%;">
> $Z_d = \prod_{j=1}^{m} Z_{dj}$
> </div>
> <br>

Initiating decision criteria and decision functions follows a similar procedure as shown before with the objective criteria. Here we instead draw on the _DecisionCriterion_ and _DecisionFunction_ classes to create the corresponding instances. A first important difference is that here we must provide 'num_coef', i.e., the __number of unspecified coefficients__ used in each decision function. During runtime, coefficient values are collected as a list of floats that is passed to the decision function as the second positional argument (the first positional argument is again the list of input layer arrays).

Secondly, when creating the DecisionCriterion instance, we must also specify 'cover_type', i.e., the __cover type__ the decision function is associated with. This will allow the decision values to be used to spatially allocate the corresponding cover type. The same cover type must also be included in the __demand__ component (see next section).

In the __code example__ below, we again draw on pre-defined example of a decision function called 'potential_sng'. Please consult [functions.py](functions.py) to see how this function is defined. Pay particular attention to how it takes both input layers and coefficient values as input, and the fact that it returns an array as output. Also note that the function definition uses several pre-defined support functions, namely _standardize_, _combine_ and _logistic_.
    

In [ ]:
from src.spatial_tool import DecisionFunction, DecisionCriterion
from problem import potential_sng

decision_criteria = []

decfun_sng = DecisionFunction(
    input_layer_ids=[
        'pasture',
        'cropland',
        'relative frequency pasture',
        'relative frequency cropland',
        'ecological connectivity insect pollinators',
        'sand fraction',
        'sand fraction squared',
        'soil organic carbon',
        'slope',
        'road access',
        'protected areas'
    ],
    num_coef=11,
    function=potential_sng
)
deccrit_sng = DecisionCriterion(
    criterion_id='potential semi-natural grassland',
    cover_type='semi-natural grassland',
    decision_function=decfun_sng
)

decision_criteria.append(deccrit_sng)


#### Demand
__Demand__ is a fairly straightforward component of the optimization problem. Essentially, it is a _dictionary_ containing _cover type labels_ as keys and integer numbers as corresponding values, with the numbers reflecting the _number of cells that must be allocated_ in the model space. Recall that we must define a _separate decision criterion_ for each cover type in demand.

To define demand, we simply draw on the built-in _Dictionary_ data type, as shown in the following short __code example__. The specified cover types must also be present in the _label map of the model space_, and the specified numbers must be _integers greater than 0_.


In [ ]:
demand = {
    'semi-natural grassland': 5000,
    'flower strips': 1000
}


#### Algorithm parameters
The SONATA spatial tool uses _Multi-Objective Differential Evolution_ (__MODE__) to perform optimization. MODE is a particular case of a Genetic Algorithm that is especially well-suited to _optimize numerical values_, here the unspecified _decision function coefficients_.

> #### Genetic Algorithms
> Genetic Algorithms are __population-based optimizers__ that use _collections of solutions_ to help explore the search space and try to converge on
> the global optimum. The population of solutions is gradually improved over the course of a certain number of _generations_. During each generation, 
> all new solutions are _evaluated_ using the objective criteria. A subset of best performing solutions, also called _parents_, is then _selected_ to 
> generate new solutions, typically called _children_. These child solutions are produced through _crossover_ i.e., by combining parts of two or more 
> parent solutions. _Mutation_ is also applied on each generated child solution, to avoid premature convergence and enhance exploration of the search 
> space.
> 
> <figure>
> <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Estimation_of_Distribution_Algorithm_animation.gif">
> <figcaption>Downloaded from <a href="https://upload.wikimedia.org/wikipedia/commons/2/2b/Estimation_of_Distribution_Algorithm_animation.gif">here</a>.</figcaption>
> </figure>


While the spatial tool has a pre-defined set of parameter configurations for MODE, making this technically an optional component of the problem definition, it is strongly advised to experiment with and __finetune these parameters__ to get better results. These parameters should always be adjusted to better fit the particulars of the optimization problem being considered (e.g., the total number of unspecified coefficients) and/or the technical specifications of the machine on which the spatial tool will be used (e.g., the number of available CPU/GPU).

As with most implementations of a Genetic Algorithm, there are a few __basic evolutionary parameters__ that can be played with. Without being exhaustive and discussing their particular roles, these include:

- The number of generations
- The number of parent solutions
- The number of child solutions
- Mutation strength
- Mutation rate
- Crossover strength
- Crossover rate

Besides the basic evolutionary parameters, the option is also provided to specify parameters that can be used to apply __adaptive learning strategies__. In short, adaptive learning strategies dynamically adjust some of the evolutionary parameters during optimization in response to changes in global or per-solution _performance statistics_, that are documented over the course of generations. This can be useful to make the optimizer _converge quicker to the global optimum_ and/or to _avoid converging prematurely to a local optimum_. Possible adaptive learning parameters include:

- Rank pressure
- Crowding pressure
- Learning rate
- Target success rate

Rather than explaning all these parameters here, we point you to the field descriptions included in [src/spatial_tool/algorithm.py](src/spatial_tool/algorithm.py).

As shown in the __code example__ below, to adjust the default MODE parameters in the spatial tool we draw on the class _Algorithm_. The resulting instance contains several attributes representing annotated and constrained algorithm parameters, with corresponding default values in case some of these parameters are not specified.


In [ ]:
from src.spatial_tool import Algorithm

algorithm = Algorithm(
    num_parent=20,
    num_offspring=20,
    generations=50,
    crossover_rate=.5,
    rank_pressure=0,
    crowding_pressure=0,
    mutation_rate=.5,
    mutation_strength=.2,
    max_mutation_strength=1.,
    min_mutation_strength=0.1,
    target_success_rate=.1,
    learning_rate=.5,
    coef_ll=-5.,
    coef_ul=5.
)


> #### Evolutionary cycles and allocation cycles
> Seeing that the key inputs of the spatial tool have now been covered, it might be a good time to share an overview of __how the spatial tool
> works under the hood__.
> 
> During runtime, the optimizer will go through an __evolutionary cycle__, using MODE with the algorithm parameters specified. For each solution
> evaluated during the evolutionary cycle, a __spatial allocation cycle__ will be completed. A _solution_ should firstly be understood here as a
> collection of _coefficient values_ for the decision functions. An allocation cycle can also be referred to as a _simulation_, i.e.,  of how the cover types
> in demand would be distributed spatially given the decision values yielded with the tested solution.
>
> During each iteration of an allocation cycle, the dynamic layers and decision values are first updated. Then a cover type in demand is allocated to the cell
> in the model space having the _highest corresponding decision value_. This results in an _adjusted model space_ that differs from its original state. Finally,
> the demand of the corresponding cover type is lowered by > 1. The allocation process is _repeated until all demand has been satistified_, at which point the
> allocation stops and a _final adjusted model space_ is obtained.
>
> The adjusted model spaces generated by the tested solutions are passed back to the evolutionary cycle, where they will then be __evaluated using the objective
> criteria__, to see how well the different solution performs. This in turn forms the basis for selection and subsequent crossover and mutation. The
> evolutionary cycle continues until the specified number of generations has been traversed. Assuming the optimization problem is multi-objective, we then end
> up with a __Pareto front of non-dominated solutions__. If we then simulate, i.e., apply the allocation cycle, on these non-dominated solutions, we get the
> corresponding adjusted model spaces, which is typically a key -if not the most important- output of optimization.
> 
> <img src="img/evolutionary_allocation_cycle.png" width="1000">

### Example use case
An __example use case__ has been prepared to perform optimization with. The coded problem definition can be found in the [problem.py](problem.py) script. The custom functions used in the problem definition are kept in a separate script called [functions.py](functions.py). The initial model space and static layers used for this optimization problem are included in the folder [data](data).

For this example use case we will try to tackle a _simplified, smaller and partially invented_ equivalent of the optimization problem envisioned in the __SONATA__ project. The __study area__ covers a rectangular area of about 15x22km located in the Vojvodina region in Serbia, North of the town of Novi Bečej. The objective is to convert a certain number of cells, let's say _100 cells_ (we can easily change this), to __Semi-Natural Grasslands (SNG)__ in this area. In other words, SNG is the __cover type in demand__. SNG are a key _Nature-based Solution_ to help boost __Crop Yield (CY) in nearby sunflower fields__ as well as to increase __Pollinator Diversity (PD)__ and __Carbon Sequestration (CS)__. These are the 3 ecosystem services, i.e. the __objective criteria__, that we want to __maximize__.

> __Semi-natural grasslands__ are important habitats for various species of insect pollinators, including hoverflies and bumblebees. Besides being key
> for __biodiversity__, these insects also provide pollination services to crop fields located in the vicinity of semi-natural grasslands. In other
> words, semi-natural grassland have a __positive effect on agricultural output__, even though they themselves can generally not be used for intensive
> agricultural production.
> 
> Semi-natural grasslands can also __sequester considerable amounts of carbon__ in their soils. And contrary to forests, which
> typically are stronger carbon sinks, semi-grasslands can be established considerably quicker. In other words, as a means to mitigate climate
> change in the short- to medium-term, semi-natural grasslands are an important measure to achieve results relatively fast, to be used alongside 
> planting forests that remain critical in the longer term.
> 
> <figure>
> <img src="https://upload.wikimedia.org/wikipedia/commons/8/89/%C5%A0ipikovo%2C_Serbia_-_panoramio.jpg" width="600">
> <figcaption>Downloaded from <a href="https://upload.wikimedia.org/wikipedia/commons/8/89/%C5%A0ipikovo%2C_Serbia_-_panoramio.jpg">here</a>.</figcaption>
> </figure>

> Note that __sunflower cultivation__ is a crucial and pervasive component of agricultural production in Vojvodina, and it is becoming even more 
> important in light of climate change because sunflowers are fairly __drought and heat stress resistant__. However, sunflowers are also 
> __strongly dependent on insect pollination__ to obtain optimal yields. These are some of the main reason why sunflower crop fields receive
> particular attention in SONATA.
> 
> <figure>
> <img src="https://upload.wikimedia.org/wikipedia/commons/e/eb/Sunflowers_helianthus_annuus.jpg" width="600">
> <figcaption>Downloaded from <a href="https://upload.wikimedia.org/wikipedia/commons/e/eb/Sunflowers_helianthus_annuus.jpg.">here</a>.</figcaption>
> </figure>

The __model space__ of the study area, included in [data/model_space.tif](data/model_space.tif), has a __spatial resolution of 100m__, meaning that _each cell has a surface area of 1 ha_, and its dimensions are __149 rows by 216 columns__. All other layers have the same resolution and dimensions The __cover types__, with corresponding (alpha)numerical labels, that can occur in the model space are:

0. Background <span style="color:gray;">(settlements, roads and any other land use not covered by the other cover types)</span>
1. Pasture
2. Semi-natural grassland
3. Forest
4. Wetland/surface water
5. Cropland <span style="color:gray;">(that is/can be used for sunflower cultivation)</span>

Besides the model space, there are __5 static layers__:

- Soil Organic Carbon (SOC)
- Sand fraction
- Slope
- Road access
- Protected Areas

In [problem.py](problem.py), you will notice that we define several types of __dynamic/function-based layers__ as well, including:

- Cover type masks
    - Obtained by applying an indicator function on the model space.
    - Shows where in the model space a specific cover type occurs.
- Relative frequencies
    - Obtained by applying a convolve function on the cover type masks.
    - Speaks to how much of a cover type can be found in the vicinity of each cell.
- Ecological connectivity for insect pollinators
    - Obtained by applying convolve on an insect habitat suitability map.
    - Insect habitat suitability is derived from the model space.
    - Note that this is only an approximated computation of actual [__ecological connectivity__](https://ecologicalconnectivity.com/connectivity).
    - Speaks to how much a cell is connected with/part of surrounding insect habitats, if present.
    
The __objective functions__ defined for the __objective criteria__ are included in [functions.py](functions.py). Please note that, for the purpose of this demonstration, we will only use __made-up functions__, that superficially reflect some basic insights on how crop yield, pollinator diversity and carbon sequestration may be affected by various environmental characteristics. The numerical values yielded by these functions __only serve an illustrative purpose__, they don't reflect realistic outcomes.

Here we will only share some general info about the defined objective functions, consult the script for the particulars:
- Crop yield
    - Could be expressed e.g. in ton/ha/year or euro/ha/year.
    - Occurs on pasture and cropland.
    - Each of these cover types has a base yield.
    - The base yield of a cell can be increased/decreased based on various environmental mutators.
- Pollinator diversity
    - Could be expressed e.g. in species/ha.
    - Occurs on every cover type except background.
    - Uses similar approach like crop yield, with base value and environmental mutators.
- Carbon sequestration
    - Could be expressed e.g. in tons of CO$_2$/ha/year.
    - Occurs on pasture, semi-natural grassland, forest and wetlands/surface waters.
    - Mostly using base value per cover type, with limited environmental mutators.
    
Since we have only 1 cover type in demand, i.e., semi-natural grassland, we only need to define __1 decision criterion__ associated with this cover type. The __decision function__ defined for this criterion is called "potential_sng", and it takes _11 input layers_ (of which 1 is purely a constraint) and _11 coefficient values_ as input. The mathematical form of the decision function basically follows the earlier shown formulas. We enforce __2 spatial constraints__ in this decision function:

1. Only pasture or cropfield cells can be converted to new semi-natural grassland.
2. Cells located in protected areas may not change cover type.


### Initiating the optimizer instance
Once the optimization problem is defined, initiating an __Optimizer__ instance with the spatial tool then becomes straightforward, as shown in the __code example__ below.

Note that when you run this code block, the spatial tool will perform a series of checks on the specified optimization problem. If there are any inconsistencies in the provided inputs (e.g. dimensions of slope layer don't match those of the model space), or if the required data formats were not respected (e.g., demand is defined as a list instead of a dictionary), this will yield an error indicating where the problem is located.

> #### Saving Optimizer instances
> Once created, a loaded Optimizer instance can be stored to disk as a pickle file, using the class method _to_pickle_. The pickle file will, by default, also contain > the cached layer values. As we will show further on, we can easily reload the saved Optimizer instance from pickle into an active code environment.
> 
> A second option to store the complete parameter configuration of a created Optimizer instance, _without including the actual layer values_, is saving it to YAML,
> which is essentially a structured text file format. This can be done with the class method _to_yaml_.

> #### Optional Optimizer arguments
> There are a few additional optional arguments that can be specified during Optimizer creation. The argument "check_dimensions", which equals True by default, 
> controls whether the spatial tool performs dimension checks on the specified layers. If the dimensions of these layers are large, this may incur somewhat longer
> load times.
> 
> The arguments labeled "update_distance" and "update_buffer" are closely linked with one another. Update distance allows you to limit how far, expressed in number of > cells, layer values must be updated around a cell whose cover type has changed. Update buffer then furthermore specifies how far, again expressed in cells,
> beyond the update distance additional cell values must be taken in account to yield correct values. Typically, update and buffer distances will be equal. The reason > why we specified them as 5 (cells of 100m) is because none of the dynamic layer used in this example have a convolutional kernel going further than 500m from the
> focal cell. These parameters effectively limit updating to the parts of the dynamic layers and decision criteria that are affected by an individual cell change. The > alternative would be having to continuously update these layers completely, which would be a waste of time and computational effort.
> 
> The parameter "log_delta_fitness", that equals False by default, allows you to log objective values as changes compared to the objective value obtained from the
> initial state of the model space.
> 
> Finally, "seed" gives you the option to set the seed number for pseudo-random number generation. While spatial allocation is a deterministic process, that will
> always yield the same results given the same starting conditions, the evolutionary cycle discussed earlier is partially a random process. Creating the initial
> population of solutions as well as performing crossover and mutation all draw on random sampling.
>
> There are more optional arguments that can be passed to Optimizer, including some that allow you to manage __using multiple CPU/GPU__ and __specifying the 
> computational backend__, but these advanced topics will not be covered in this introductory demonstration.


In [ ]:
from src.spatial_tool import Optimizer
from problem import layers, objective_criteria, decision_criteria, demand, algorithm

optimizer = Optimizer(
    layers=layers,  # mandatory inputs
    objective_criteria=objective_criteria,
    decision_criteria=decision_criteria,
    demand=demand,
    algorithm=algorithm,  # optional inputs
    check_dimensions=True,
    update_distance=5,
    update_buffer=5,
    log_delta_fitness=True,
    seed=42
)

# save optimizer instance as pickle file (includes the cached layer values by default)
optimizer.to_pickle('optimizer.pkl')

# save optimizer configuration as YAML file (not including the cached layer values)
optimizer.to_yaml('optimizer.yaml')


### Running the optimization process
Assuming all went well during Optimizer instance creation, which would mean that the given optimization problem definition is at least technically viable, we need little additional code to actually __launch the optimization process__. For this, it suffices to call the class method _optimize_.

While this step seems concise, at least in code needed from the user, _optimize_ will typically entail a __considerable computation effort__, possibly requiring _hours or even days_ to complete for a real optimization problem. Runtimes strongly depend on the dimensions of the used layers, the complexity of the layer/criterion functions, the specified cover type demand and the number of CPUs/GPUs made available to run the spatial tool. Normally, for the small and simplified optimization problem considered in this demo, runtimes should be manageable.

Once the optimization has been completed, you'll likely want to store the "calibrated" Optimizer instance to disk again, possibly using an adapted file name like shown below. A calibrated optimizer instance contains the __non-dominated solutions__ obtained at the end of the evolutionary cycle. It will also contain __optimization performance statistics__ collected throughout the evolutionary cycle, i.e., aggregated statistics derived from the populations, showing how the objective values evolve over the course of the different generations.


In [ ]:
import pickle

with open('optimizer.pkl', 'rb') as f:
    optimizer = pickle.load(f)

optimizer.optimize()

optimizer.to_pickle('optimizer_calibrated.pkl')


### Performing analysis on/with the calibrated optimizer
Again assuming all went well up to this point, we arrive at the final step in the use of the spatial tool, namely analysis. While there are definitely many more possibilities than shown here, the code examples below do cover __3 types of analyses__ that most spatial tool users will likely want to perform on or with their hard-earned calibrated Optimizer instance.

#### Learning curve
A common analysis performed in Genetic Algorithm-based Optimization, as well as in Machine Learning, entails the production and inspection of the so-called [__Learning Curve__](https://en.wikipedia.org/wiki/Learning_curve_(machine_learning)). A learning curve is a two-dimensional plot in which the vertical axis typically reflects one or more performance statistics (_here objective criteria_) and the horizontal axis the amount of effort invested to obtain the corresponding result (_here generations passed in the evolutionary cycle_).

Firstly, a learning curve should tell us something about the __performance__ of the optimizer, i.e., showing how well the optimizer managed to maximize/minimize the objective criteria, as seen over the different objective values achieved in a population during each generation. Secondly, a learning curve also speaks to the __convergence__ of the optimizer. If there is still a lot of variation in the final parts of the lines shown in the learning curve, this likely indicates that the optimization hasn't yet reached the global optimum and that maybe we should use a larger number of generations and/or employ adaptive learning strategies to converge faster.

In the __code example__ below we illustrate how to construct a composed learning curve plot, showing the learning curves for each objective criterion and an overall weighted sum of the objective criteria in separate subplots. We will not explain the complete code, as most of it basically handles the gathering and plot design. Do note the part of the code where the optimization performance statistics are extracted from the calibrated optimizer instance, using the _Optimizer_ class methode _get_performance_statistics_.


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import ListedColormap, BoundaryNorm


# specify some parameters
objective_criteria = [
    'agricultural yield',
    'carbon sequestration',
    'pollinator diversity'
]
labels = objective_criteria + ['overall']
labels = [l[0].upper() + l[1:] for l in labels]
colors = ['orange', 'blue', 'green', 'gray']

labels_best_solutions = ['Best ' + objcrit for objcrit in objective_criteria]
labels_best_solutions += ['Best weighted solution']

value_colors = {
    0: '#e3e3e3',  # background
    1: "#98b99b",  # pasture
    2: "#30ce6f",  # semi-natural grassland
    3: "#027800",  # forest
    4: "#259ad5",  # wetland/surface water
    5: "#d4cc56"   # cropfield
}
cmap = ListedColormap([value_colors[v] for v in value_colors.keys()])
norm = BoundaryNorm(np.arange(-0.5, len(value_colors) + 0.5, 1), cmap.N)

nbs_labels = [
    'semi-natural grassland',
    'flower strip'
]


# get the calibrated optimizer
with open('optimizer_calibrated.pkl', 'rb') as f:
    optimizer = pickle.load(f)

generations = np.arange(1, optimizer.algorithm.generations + 1)


# collect the performance statistics
means = []
minvals = []
maxvals = []

for objective_criterion in objective_criteria:

    avg = np.array(optimizer.get_performance_statistics(
        objective_criterion=objective_criterion,
        statistic='avg'
    ))
    minval = np.array(optimizer.get_performance_statistics(
        objective_criterion=objective_criterion,
        statistic='min'
    ))
    maxval = np.array(optimizer.get_performance_statistics(
        objective_criterion=objective_criterion,
        statistic='max'
    ))

    # normalize the performance statistics
    mn = min(minval)
    mx = max(maxval)
    minval = (minval - mn) / (mx - mn)
    maxval = (maxval - mn) / (mx - mn)
    avg = (avg - mn) / (mx - mn)

    means.append(avg)
    minvals.append(minval)
    maxvals.append(maxval)


# get overal normalized performance statistics
avg_all = sum(means) / 3.0
minvals_all = sum(minvals) / 3.0
maxvals_all = sum(maxvals) / 3.0

mn = min(minvals_all)
mx = max(maxvals_all)
avg_all = (avg_all - mn) / (mx - mn)
minvals_all = (minvals_all - mn) / (mx - mn)
maxvals_all = (maxvals_all - mn) / (mx - mn)

means.append(avg_all)
minvals.append(minvals_all)
maxvals.append(maxvals_all)


# plot the performance statistics
plt.clf()
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

for i, ax in enumerate(axes):

    mean = means[i]
    minval = minvals[i]
    maxval = maxvals[i]
    color = colors[i]
    label = labels[i]

    ax.plot(generations, maxval,
        color=color,
        linestyle='--',
        label='maximum'
    )
    ax.plot(generations, mean,
        color=color,
        label='mean'
    )
    ax.plot(generations, minval,
        color=color,
        linestyle=':',
        label='minimum'
    )

    ax.fill_between(generations, minval, maxval,
        color=color,
        alpha=0.3
    )

    ax.set_title(label)
    if i in [2, 3]:
        ax.set_xlabel("Generations")
    if i in [0, 2]:
        ax.set_ylabel("Performance (normalised score)")

    ax.xaxis.set_major_locator(MultipleLocator(5))   # major ticks every 5 generations
    ax.xaxis.set_minor_locator(MultipleLocator(1))   # minor ticks every generation

    ax.grid(which='major', linestyle='-', linewidth=0.8, alpha=0.7)
    ax.grid(which='minor', linestyle='--', linewidth=0.5, alpha=0.3)

    if len(generations) <= 10:
        ax.tick_params(which="both", labelbottom=True)

    if i == 3:
        ax.legend(
            loc='lower right',
            fontsize=8
        )

    ax.set_facecolor("whitesmoke")


# show the final plot
plt.tight_layout()
plt.show()


#### Best per-criterion/weighted solution
In this second analysis, we will simulate and plot the adjusted model spaces obtained with the __3 solutions scoring best on each separate objective criterion__. Additionally, will also get and plot the adjusted model space of the __best weighted solution__.

Just as before, we won't be explaining the full script shown in the __code example__ below. Instead we just point out the main steps.

First, the __best per-criterion solutions__ are gathered using the _Optimizer_ class method _get_best_solution_. Similarly, we obtain the __best weighted solution__ using the class method _get_best_weighted_solutions_. Note that we could play around with the weights used to obtained the solution, but for now we will stick with the default values (i.e., equal weights for each objective criterion).

In a second step we then __simulate__ these solutions, using the _Optimizer_ class method _simulate_. Note that _simulate_ takes one argument, being the solution with which simulation is to be performed. Afterwards, we can get the __array of the adjusted model space__ using the method _get_model_space_.

Thirdly, we __identify where cells have been changed__ in each solution. For this we compare each adjusted model space with the __initial state of the model space__. In case we have to revert an adjusted model space, we use the _Optimizer_ class method _reset_model_space_values_. The comparison of adjusted and original model space allows us to derive the image coordinates of cells that have been converted to semi-natural grassland. These locations are then plotted on top of the base map formed by the original model space to visualize the simulated changes.


In [ ]:
import pickle
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.gridspec as gridspec


# specify some parameters
objective_criteria = [
    'agricultural yield',
    'carbon sequestration',
    'pollinator diversity'
]
labels = objective_criteria + ['overall']
labels = [l[0].upper() + l[1:] for l in labels]
colors = ['orange', 'blue', 'green', 'gray']

labels_best_solutions = ['Best ' + objcrit for objcrit in objective_criteria]
labels_best_solutions += ['Best weighted solution']

value_colors = {
    0: '#e3e3e3',  # background
    1: "#98b99b",  # pasture
    2: "#30ce6f",  # semi-natural grassland
    3: "#027800",  # forest
    4: "#259ad5",  # wetland/surface water
    5: "#d4cc56"  # cropfield
}
cmap = ListedColormap([value_colors[v] for v in value_colors.keys()])
norm = BoundaryNorm(np.arange(-0.5, len(value_colors) + 0.5, 1), cmap.N)


# get the calibrated optimizer
with open('optimizer_calibrated.pkl', 'rb') as f:
    optimizer = pickle.load(f)
    
    
# collect the best solutions per criterion and a weighted sum of the criteria
best_ay = optimizer.get_best_solution('agricultural yield')
best_cs = optimizer.get_best_solution('carbon sequestration')
best_pd = optimizer.get_best_solution('pollinator diversity')
best_weighted = optimizer.get_best_weighted_solution()

best_solutions = [
    best_ay,
    best_cs,
    best_pd,
    best_weighted
]


# simulate the best solutions to get their adjusted model spaces
iterator = list(enumerate(best_solutions))
best_solution_arrays = []

for s, solution in tqdm(iterator, desc='Simulating best solutions...'):

    optimizer.simulate(solution)
    ms_lay = optimizer.get_model_space()
    best_solution_arrays.append(ms_lay.get())


# get the original model space values
optimizer.reset_model_space_values()
lay_ms_start = optimizer.get_model_space()
ms_start = lay_ms_start.get()


# get the original locations of the NbS semi-natural grassland
sng_start = np.where(ms_start == 2, 1, 0)


# get the cover type label map
label_map = lay_ms_start.label_map
label_map_inv = {label: key for key, label in label_map.items()}


# plot the new NbS locations
plt.clf()
fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 0.1])
axes = [fig.add_subplot(gs[i]) for i in range(4)]

for l, label in enumerate(labels_best_solutions):

    # get the simulated model space
    ms_adj = best_solution_arrays[l]

    # get the new locations of SNG
    sng_adj = np.where(ms_adj == 2, 1, 0)
    diff_sng = sng_adj - sng_start
    ind_sng = np.where(diff_sng != 0)

    # plot allocations over the initial model space
    ax = axes[l]
    ax.axis('off')
    ax.set_facecolor("whitesmoke")
    ax.set_aspect("equal")
    ax.set_title(label)

    im = ax.imshow(ms_start,
        cmap=cmap,
        norm=norm,
        origin="upper",
        interpolation="nearest"
    )

    ax.scatter(ind_sng[1], ind_sng[0],
        marker="^",
        s=20,
        facecolors="none",
        edgecolors="black",
        linewidths=1.,
        alpha=0.5
    )


# plot the legend
ax_legend = fig.add_subplot(gs[4:])
ax_legend.axis('off')

legend_elements = []

for v, color in value_colors.items():

    label = label_map_inv[v]
    label = label[0].upper() + label[1:]
    legend_elements.append(Patch(
        facecolor=color,
        edgecolor="black",
        label=label
    ))

legend_elements.append(Line2D([], [],
    marker="^",
    linestyle="None",
    markersize=10,
    markerfacecolor="none",
    markeredgecolor="black",
    label="New semi-natural grassland"
))

ax_legend.legend(
    handles=legend_elements,
    loc="center",
    frameon=True,
    ncol=3
)


# show the final plot
plt.tight_layout()
plt.show()


#### Pareto frequency map
TBC
...